In [25]:
import pandas as pd
import json

test_json_path = "/opt/ml/detection/dataset/test.json"
submission_file_path = "/opt/ml/detection/mmdetection_new/boostcamp/swinB1-5_B_U_yR_submission.csv"
pseudolabel_file_path = '/opt/ml/detection/dataset/pseudo_test.json'

with open(test_json_path, "r") as test_json:
    test_dict = json.load(test_json)

pseudo_dict = {}

pseudo_dict["info"] = test_dict["info"]
pseudo_dict["licenses"] = test_dict["licenses"]
pseudo_dict["images"] = test_dict["images"]
pseudo_dict["categories"] = test_dict["categories"]
pseudo_dict["annotations"] = test_dict["annotations"]

submission_file = pd.read_csv(submission_file_path)

cnt = 0 
for idx, row in submission_file.iterrows():
    row_unit = row["PredictionString"].split(" ")
    for j in range(0, len(row_unit)-6, 6):
        if float(row_unit[j+1]) > 0.5:
            category_id = row_unit[j]
            image_id = idx
            bbox = [row_unit[j+2], row_unit[j+3],row_unit[j+4],row_unit[j+5]]
            for idx in range(len(bbox)):
                bbox[idx] = round(float(bbox[idx]),2)
            is_crowd = 0
            q = {'image_id':image_id, "category_id":int(category_id), "area" : round(float(row_unit[j+4]) * float(row_unit[j+5]),2), "iscrowd": 0, "id":cnt, "bbox":bbox}
            pseudo_dict["annotations"].append(q)
            cnt += 1

            

In [26]:
pseudo_dict['annotations'][0]

{'image_id': 0,
 'category_id': 7,
 'area': 975352.86,
 'iscrowd': 0,
 'id': 0,
 'bbox': [603.91, 521.07, 952.65, 1023.83]}

In [28]:
with open(pseudolabel_file_path, 'w', encoding='utf-8') as f:
    json.dump(pseudo_dict, f, ensure_ascii=False, indent=4)
    
submission_file.head()

,image_id,PredictionString
0,test/0000.jpg,7 0.9907011389732361 603.9096069335938 521.074...
1,test/0001.jpg,5 0.7696418166160583 131.93116760253906 0.2005...
2,test/0002.jpg,1 0.9478424787521362 774.7786865234375 414.753...
3,test/0003.jpg,9 0.8673842549324036 154.95582580566406 264.45...
4,test/0004.jpg,1 0.8048049211502075 207.42279052734375 258.55...
